In [1]:
import csv
import tensorflow as tf
import tensorflow_hub as hub
from glob import glob
import os

/home/yeakiest/.pyenv/versions/3.6.15/envs/.tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yeakiest/.pyenv/versions/3.6.15/envs/.tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yeakiest/.pyenv/versions/3.6.15/envs/.tensorflow1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [2]:
tf.test.is_gpu_available()

False

In [3]:
module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/2")
height, width = hub.get_expected_image_size(module)
ch = 1
batch_size = 10

path_to_indir = "/mnt/g/20240126_thunder_Sporulation_Cdc3-intEnvy_YSIY874_YSIY1447_YSIY1448/Project000_YSIY874/sum_crop/"
image_list = glob(os.path.join(path_to_indir, "*.png"))
# image_list = tf.convert_to_tensor(['images/img_{:04d}.jpg'.format(i+1) for i in range(2500)])
image_list = tf.convert_to_tensor(image_list)
input_queue = tf.train.slice_input_producer([image_list], num_epochs=1, shuffle=False)

image_bytes = tf.read_file(input_queue[0])
image = tf.image.decode_image(image_bytes, channels=ch)
image = tf.image.resize_bilinear([image], [height, width])
image = tf.reshape(image, tf.stack([height, width, ch]))
image = tf.image.grayscale_to_rgb(image)  # Convert grayscale to RGB
images = tf.train.batch([image], batch_size=batch_size, allow_smaller_final_batch=True)


features = module(images)

with open('feature_vecs.tsv', 'w') as fw:
    csv_writer = csv.writer(fw, delimiter='\t')
    with tf.Session() as sess:
        sess.run([tf.local_variables_initializer(),
                  tf.global_variables_initializer()])
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            while not coord.should_stop():
                fvecs = sess.run(features)
                csv_writer.writerows(fvecs)
        except:
            pass
        finally:
            coord.request_stop()
            coord.join(threads)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/1ae09af0a58a5ec02e9cbde4f265b10eafbf79e6/variables/variables' with MobilenetV2/Conv/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/1ae09af0a58a5ec02e9cbde4f265b10eafbf79e6/variables/variables' with MobilenetV2/Conv/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/1ae09af0a58a5ec02e9cbde4f265b10eafbf79e6/variables/variables' with MobilenetV2/Conv/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/Conv/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/1ae09af0a58a5ec02e9cbde4f265b10eafbf79e6/variables/variables' with MobilenetV2/Conv/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/Mobilene

In [ ]:
#　最後にこれを実行する
# tensorboard --logdir .